In [2]:
# Stop warnings
import warnings
warnings.filterwarnings("ignore")

# Debug
import ipdb
deb = ipdb.set_trace

# General imports
import os
import sys
import json
import pandas as pd
import plotly.io as pio

# Personal import
sys.path.append("{}/../../analysis_code/utils".format(os.getcwd()))
from plot_utils import *

In [3]:
# Inputs
# Directories
main_dir = '/Users/uriel/disks/meso_shared'
project_dir = 'RetinoMaps'
format_ = 'fsnative'
extension ='.func.gii'
subject = 'group'
suffix = 'Sac_Pur'

In [4]:
# Load settings
with open('../settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)
rois = analysis_info['rois']


# Figures settings
with open('../figure_settings.json') as f:
    json_s = f.read()
    figure_info = json.loads(json_s)
num_ecc_size_bins = figure_info['num_ecc_size_bins']
num_ecc_pcm_bins = figure_info['num_ecc_pcm_bins']
num_polar_angle_bins = figure_info['num_polar_angle_bins']
max_ecc = figure_info['max_ecc']
screen_side = figure_info['screen_side']
gaussian_mesh_grain = figure_info['gaussian_mesh_grain']
hot_zone_percent = figure_info['hot_zone_percent']
plot_groups = figure_info['plot_groups']
fig_width = figure_info['fig_width']
categories_to_plot = figure_info['categories_to_plot']

In [5]:
# Figure settings
roi_colors = {'V1': 'rgb(243, 231, 155)', 
              'V2': 'rgb(250, 196, 132)', 
              'V3': 'rgb(248, 160, 126)', 
              'V3AB': 'rgb(235, 127, 134)', 
              'LO': 'rgb(150, 0, 90)',  
              'VO': 'rgb(0, 0, 200)', 
              'hMT+': 'rgb(0, 25, 255)', 
              'iIPS': 'rgb(0, 152, 255)', 
              'sIPS': 'rgb(44, 255, 150)', 
              'iPCS': 'rgb(151, 255, 0)', 
              'sPCS': 'rgb(255, 234, 0)', 
              'mPCS': 'rgb(255, 111, 0)'}



In [6]:
# Create folders and fns
fig_dir = '/Users/uriel/Downloads/test'
tsv_dir = '{}/{}/derivatives/pp_data/{}/{}/prf/tsv'.format(
    main_dir, project_dir, subject, format_)

In [ ]:

# Create folders and fns
fig_dir = '{}/{}/derivatives/pp_data/{}/{}/intertask/figures'.format(
    main_dir, project_dir, subject, format_)

tsv_dir = '{}/{}/derivatives/pp_data/{}/{}/intertask/tsv'.format(
    main_dir, project_dir, subject, format_)

figures = {}
# loop over categories
for categorie_to_plot in categories_to_plot : 
    print(categorie_to_plot)
    figures[categorie_to_plot] = []
    fig_dir_categorie = '{}/{}/derivatives/pp_data/{}/{}/intertask/figures/{}'.format(
        main_dir, project_dir, subject, format_, categorie_to_plot)
    os.makedirs(fig_dir_categorie, exist_ok=True)

    tsv_dir_categorie = '{}/{}/derivatives/pp_data/{}/{}/intertask/tsv/tsv_{}'.format(
        main_dir, project_dir, subject, format_, categorie_to_plot)

    # Violins plot
    tsv_violins_fn = "{}/{}_{}_prf_violins_{}.tsv".format(tsv_dir_categorie, subject, categorie_to_plot, suffix)
    df_violins = pd.read_table(tsv_violins_fn, sep="\t")
    fig_violins = prf_violins_plot(df_violins=df_violins, fig_width=fig_width, fig_height=600, 
                            rois=rois, roi_colors=roi_colors)
    figures[categorie_to_plot].append(fig_violins)



    # Ecc.size plots
    tsv_ecc_size_fn = "{}/{}_{}_prf_ecc_size_{}.tsv".format(tsv_dir_categorie, subject, categorie_to_plot, suffix)
    df_ecc_size = pd.read_table(tsv_ecc_size_fn, sep="\t")
    fig_ecc_size = prf_ecc_size_plot(df_ecc_size=df_ecc_size, fig_width=fig_width, 
                            fig_height=400, rois=rois, roi_colors=roi_colors,
                            plot_groups=plot_groups, max_ecc=max_ecc)
    figures[categorie_to_plot].append(fig_ecc_size)

    # Ecc.pCM plot
    tsv_ecc_pcm_fn = "{}/{}_{}_prf_ecc_pcm_{}.tsv".format(tsv_dir_categorie, subject, categorie_to_plot, suffix)
    df_ecc_pcm = pd.read_table(tsv_ecc_pcm_fn, sep="\t")
    fig_fn = "{}/{}_{}_prf_ecc_pcm_{}.pdf".format(fig_dir_categorie, subject, categorie_to_plot, suffix)
    fig_ecc_size = prf_ecc_pcm_plot(df_ecc_pcm=df_ecc_pcm, fig_width=fig_width, fig_height=400, 
                            rois=rois, roi_colors=roi_colors,
                            plot_groups=plot_groups, max_ecc=max_ecc)
    figures[categorie_to_plot].append(fig_ecc_size)


    # Polar angle distributions
    tsv_contralaterality_fn = "{}/{}_{}_prf_contralaterality_{}.tsv".format(tsv_dir_categorie, subject, categorie_to_plot, suffix)
    df_contralaterality = pd.read_table(tsv_contralaterality_fn, sep="\t")
    tsv_polar_angle_fn = "{}/{}_{}_prf_polar_angle_{}.tsv".format(tsv_dir_categorie, subject, categorie_to_plot, suffix)
    df_polar_angle = pd.read_table(tsv_polar_angle_fn, sep="\t")
    figs, hemis = prf_polar_angle_plot(df_polar_angle=df_polar_angle, fig_width=fig_width, 
                                        fig_height=300, rois=rois, roi_colors=roi_colors,
                                        num_polar_angle_bins=num_polar_angle_bins)
    for (fig, hemi) in zip(figs, hemis):
        if hemi == 'hemi-LR':
            fig_polar = fig
    figures[categorie_to_plot].append(fig_polar)

    # Contralaterality plots
    tsv_distribution_fn = "{}/{}_{}_prf_distribution_{}.tsv".format(tsv_dir_categorie, subject, categorie_to_plot, suffix)
    df_distribution = pd.read_table(tsv_distribution_fn, sep="\t")
    fig_fn = "{}/{}_{}_contralaterality_{}.pdf".format(fig_dir_categorie, subject, categorie_to_plot, suffix)
    fig_contralaterality = prf_contralaterality_plot(df_contralaterality=df_contralaterality, 
                                    fig_width=fig_width, fig_height=300, 
                                    rois=rois, roi_colors=roi_colors)
    figures[categorie_to_plot].append(fig_contralaterality)


    # Spatial distibution plot
    figs, hemis = prf_distribution_plot(df_distribution=df_distribution, 
                                        fig_width=fig_width, fig_height=300, 
                                        rois=rois, roi_colors=roi_colors, screen_side=screen_side)

    for (fig, hemi) in zip(figs, hemis):
        if hemi == 'hemi-LR':
            fig_spatial_dist = fig
            
    figures[categorie_to_plot].append(fig_spatial_dist)



vision
vision_and_pursuit_and_saccade


In [ ]:
len(html_figures_all)

In [ ]:
figs_dict = figures

# Liste des titres pour chaque figure (tous les titres sont déjà prévus pour 16 figures)
figure_titles = [
    "active vertex", 
    "violins plot", 
    "median plot", 
    "ecc size relation",
    "pCM size relation", 
    "Polar angle distribution", 
    "Contralaterality", 
    "pRF distribution",
    "figure 9 title", 
    "figure 10 title", 
    "figure 11 title", 
    "figure 12 title", 
    "figure 13 title", 
    "figure 14 title", 
    "figure 15 title", 
    "figure 16 title"
]


# Dossier de destination
output_path = "/Users/uriel/Downloads/mes_figures_plotly.html"

# Convertir chaque figure en HTML pour chaque catégorie
html_figures_all = []
for key, figs in figs_dict.items():
    for i, fig in enumerate(figs):
        html = pio.to_html(
            fig,
            full_html=False,
            include_plotlyjs='cdn' if i == 0 else False,
            config={
                'scrollZoom': False,  # Activer le zoom avec la molette
                'displayModeBar': True,  # Afficher la barre d'outils de contrôle
                'modeBarButtonsToRemove': ['zoomIn2d', 'zoomOut2d'],  # Optionnel : retirer certains boutons si tu veux
                'displaylogo': False  # Retirer le logo Plotly (facultatif)
            }
        )
        
        # Ajouter un titre avant chaque figure
        figure_html = f"<h2 style='text-align: center;'>{figure_titles[i]}</h2>" + f"<div class='plot'>{html}</div>"
        html_figures_all.append({
            "key": key, 
            "figure_html": figure_html
        })

# Assemblage HTML complet avec menu déroulant pour sélectionner les figures
full_html = f"""
<html>
<head>
    <title>Figures de {subject}</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{
            font-family: sans-serif;
            margin: 20px;
        }}
        .container {{
            display: flex;
            flex-direction: column;  /* Aligner les figures verticalement */
            gap: 30px;  /* Espacement entre les figures */
        }}
        .plot {{
            width: 80%;  /* Chaque figure prend 80% de la largeur */
            margin-left: auto;
            margin-right: auto;
        }}
        #menu {{
            position: absolute;
            top: 20px;
            right: 20px;
            padding: 10px;
            background-color: #f0f0f0;
            border-radius: 5px;
        }}
        select {{
            padding: 5px;
            font-size: 16px;
        }}
        @media (max-width: 768px) {{
            .plot {{
                width: 100%;  /* Sur mobile, les figures occupent toute la largeur */
            }}
        }}
    </style>
    <script>
        function showFigures(option) {{
            var allFigures = document.querySelectorAll('.figure');
            for (var i = 0; i < allFigures.length; i++) {{
                allFigures[i].style.display = 'none';
            }}
            
            var figuresToShow = document.querySelectorAll('.figure.' + option);
            for (var i = 0; i < figuresToShow.length; i++) {{
                figuresToShow[i].style.display = 'block';
            }}
        }}
    </script>
</head>
<body>
    <h1 style="text-align: center;">Figures de {subject}</h1>
    
    <!-- Menu déroulant dynamique basé sur les clés du dictionnaire -->
    <div id="menu">
        <select onchange="showFigures(this.value)">
            {"".join([f"<option value='{key}'>{key.replace('_', ' ').title()}</option>" for key in figs_dict.keys()])}
        </select>
    </div>

    <div class="container">
        <!-- Figures en fonction des clés du dictionnaire -->
        {"".join([f"<div class='figure {key}' style='display: none;'>{fig['figure_html']}</div>" for fig in html_figures_all for key in figs_dict.keys() if fig['key'] == key])}
    </div>
</body>
</html>
"""

# Sauvegarde dans le dossier Downloads
with open(output_path, "w") as f:
    f.write(full_html)

print(f"✅ Page HTML enregistrée dans : {output_path}")

In [15]:

import os

# Liste de figures déjà créées (ajoute ici les figures de 9 à 16 après)
figs_all = [fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8, fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8]

# Liste des titres pour chaque figure
figure_titles = [
    "active vertex", 
    "violins plot", 
    "median plot", 
    "ecc size relation",
    "pCM size relation", 
    "Polar angle distribution", 
    "Contralaterality", 
    "pRF distribution",
    "figure 9 title", 
    "figure 10 title", 
    "figure 11 title", 
    "figure 12 title", 
    "figure 13 title", 
    "figure 14 title", 
    "figure 15 title", 
    "figure 16 title"
]

# Valeur de subject (remplacer par celle que tu souhaites)
subject = "NomDuSujet"

# Dossier de destination
output_path = "/Users/uriel/Downloads/mes_figures_plotly.html"

# Convertir chaque figure en HTML (Plotly.js uniquement pour la première)
html_figures_all = []
for i, fig in enumerate(figs_all):
    html = pio.to_html(
        fig,
        full_html=False,
        include_plotlyjs='cdn' if i == 0 else False,
        config={
            'scrollZoom': False,  # Activer le zoom avec la molette
            'displayModeBar': True,  # Afficher la barre d'outils de contrôle
            'modeBarButtonsToRemove': ['zoomIn2d', 'zoomOut2d'],  # Optionnel : retirer certains boutons si tu veux
            'displaylogo': False  # Retirer le logo Plotly (facultatif)
        }
    )
    
    # Ajouter un titre avant chaque figure
    figure_html = f"<h2 style='text-align: center;'>{figure_titles[i]}</h2>" + f"<div class='plot'>{html}</div>"
    html_figures_all.append(figure_html)

# Assemblage HTML complet avec menu déroulant pour sélectionner les figures
full_html = f"""
<html>
<head>
    <title>Figures de {subject}</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{
            font-family: sans-serif;
            margin: 20px;
        }}
        .container {{
            display: flex;
            flex-direction: column;  /* Aligner les figures verticalement */
            gap: 30px;  /* Espacement entre les figures */
        }}
        .plot {{
            width: 80%;  /* Chaque figure prend 80% de la largeur */
            margin-left: auto;
            margin-right: auto;
        }}
        #menu {{
            position: absolute;
            top: 20px;
            right: 20px;
            padding: 10px;
            background-color: #f0f0f0;
            border-radius: 5px;
        }}
        select {{
            padding: 5px;
            font-size: 16px;
        }}
        @media (max-width: 768px) {{
            .plot {{
                width: 100%;  /* Sur mobile, les figures occupent toute la largeur */
            }}
        }}
    </style>
    <script>
        function showFigures(option) {{
            var allFigures = document.querySelectorAll('.figure');
            for (var i = 0; i < allFigures.length; i++) {{
                allFigures[i].style.display = 'none';
            }}
            
            if (option === 'all_vertex') {{
                var figuresToShow = document.querySelectorAll('.figure.all_vertex');
                for (var i = 0; i < figuresToShow.length; i++) {{
                    figuresToShow[i].style.display = 'block';
                }}
            }} else if (option === 'visual_vertex') {{
                var figuresToShow = document.querySelectorAll('.figure.visual_vertex');
                for (var i = 0; i < figuresToShow.length; i++) {{
                    figuresToShow[i].style.display = 'block';
                }}
            }}
        }}
    </script>
</head>
<body>
    <h1 style="text-align: center;">Figures de {subject}</h1>
    
    <!-- Menu déroulant -->
    <div id="menu">
        <select onchange="showFigures(this.value)">
            <option value="all_vertex">All Vertex</option>
            <option value="visual_vertex">Visual Vertex</option>
        </select>
    </div>

    <div class="container">
        <!-- Figures "All Vertex" (1 à 8) -->
        {"".join([f"<div class='figure all_vertex' style='display: block;'>{html_figures_all[i]}</div>" for i in range(8)])}

        <!-- Figures "Visual Vertex" (9 à 16) -->
        {"".join([f"<div class='figure visual_vertex' style='display: none;'>{html_figures_all[i]}</div>" for i in range(8, 16)])}
    </div>
</body>
</html>
"""

# Sauvegarde dans le dossier Downloads
with open(output_path, "w") as f:
    f.write(full_html)

print(f"✅ Page HTML enregistrée dans : {output_path}")

NameError: name 'fig1' is not defined